# ArcLLM Step 2: Config Loading

This notebook walks through everything built in Step 2 — the **config layer** that drives all behavior via TOML files.

**What was built:**
- Global `config.toml` (defaults + module toggles)
- Per-provider TOML files (`providers/anthropic.toml`, `providers/openai.toml`)
- Pydantic config models (`GlobalConfig`, `ProviderConfig`, `ModelMetadata`, etc.)
- `load_global_config()` and `load_provider_config()` loaders
- Path traversal prevention (NIST 800-53 AC-3)

**Why it matters:** Config-driven means agents can switch providers, adjust parameters, and toggle modules without changing code. And validated-on-load means misconfiguration crashes at startup, not during a critical LLM call.

In [ ]:
from arcllm import (
    load_global_config, load_provider_config,
    GlobalConfig, ProviderConfig,
    DefaultsConfig, ProviderSettings, ModelMetadata, ModuleConfig,
    ArcLLMConfigError,
)
import json
print("All config imports successful!")

---
## 1. Global Config

The global `config.toml` lives at `src/arcllm/config.toml`. It holds:
- **Defaults**: provider, temperature, max_tokens
- **Module toggles**: which optional features are enabled

Let's load it and explore.

In [ ]:
# Load global config — validated on load, fails fast on any issue
global_config = load_global_config()
print(f"Type: {type(global_config).__name__}")
print(f"\nDefaults:")
print(f"  Provider:    {global_config.defaults.provider}")
print(f"  Temperature: {global_config.defaults.temperature}")
print(f"  Max tokens:  {global_config.defaults.max_tokens}")

In [ ]:
# Explore modules — all disabled by default (opt-in complexity)
print(f"Modules ({len(global_config.modules)}):")
for name, module in global_config.modules.items():
    print(f"  {name:15s}  enabled={module.enabled}")

In [ ]:
# Modules can have extra fields — these are preserved via Pydantic's extra='allow'
# This lets each module define its own settings without changing the config schema
budget = global_config.modules["budget"]
print(f"Budget module:")
print(f"  enabled: {budget.enabled}")
print(f"  monthly_limit_usd: {budget.monthly_limit_usd}")

retry = global_config.modules["retry"]
print(f"\nRetry module:")
print(f"  enabled: {retry.enabled}")
print(f"  max_retries: {retry.max_retries}")
print(f"  backoff_base_seconds: {retry.backoff_base_seconds}")

fallback = global_config.modules["fallback"]
print(f"\nFallback module:")
print(f"  enabled: {fallback.enabled}")
print(f"  chain: {fallback.chain}")

rate_limit = global_config.modules["rate_limit"]
print(f"\nRate limit module:")
print(f"  enabled: {rate_limit.enabled}")
print(f"  requests_per_minute: {rate_limit.requests_per_minute}")

In [ ]:
# Full dump to see everything at once
print(json.dumps(global_config.model_dump(), indent=2, default=str))

---
## 2. Provider Config

Each provider has its own TOML file under `src/arcllm/providers/`.

A provider config has:
- **`[provider]`** section — connection settings (URL, API key env var, defaults)
- **`[models.*]`** sections — per-model metadata (context window, costs, capabilities)

In [ ]:
# Load Anthropic provider config
anthropic = load_provider_config("anthropic")
print(f"Type: {type(anthropic).__name__}")
print(f"\nProvider settings:")
print(f"  API format:   {anthropic.provider.api_format}")
print(f"  Base URL:     {anthropic.provider.base_url}")
print(f"  API key env:  {anthropic.provider.api_key_env}")
print(f"  Default model: {anthropic.provider.default_model}")
print(f"  Default temp:  {anthropic.provider.default_temperature}")

In [ ]:
# Explore model metadata
print(f"Available models ({len(anthropic.models)}):")
for model_name, meta in anthropic.models.items():
    print(f"\n  {model_name}:")
    print(f"    Context window:   {meta.context_window:,} tokens")
    print(f"    Max output:       {meta.max_output_tokens:,} tokens")
    print(f"    Supports tools:   {meta.supports_tools}")
    print(f"    Supports vision:  {meta.supports_vision}")
    print(f"    Supports thinking:{meta.supports_thinking}")
    print(f"    Input modalities: {meta.input_modalities}")
    print(f"    Cost (input):     ${meta.cost_input_per_1m}/1M tokens")
    print(f"    Cost (output):    ${meta.cost_output_per_1m}/1M tokens")

In [ ]:
# Compare with OpenAI provider
openai = load_provider_config("openai")
print(f"OpenAI provider settings:")
print(f"  API format:    {openai.provider.api_format}")
print(f"  Base URL:      {openai.provider.base_url}")
print(f"  API key env:   {openai.provider.api_key_env}")
print(f"  Default model: {openai.provider.default_model}")

print(f"\nModels:")
for model_name, meta in openai.models.items():
    print(f"  {model_name}: {meta.context_window:,} context, ${meta.cost_input_per_1m}/1M in")

### Comparing Providers

Because everything is normalized to the same types, you can compare across providers easily.

In [ ]:
# Side-by-side model comparison
print(f"{'Model':<35} {'Context':>10} {'Max Out':>10} {'$/1M In':>10} {'$/1M Out':>10} {'Tools':>6}")
print("-" * 85)

for provider_name, provider_config in [("anthropic", anthropic), ("openai", openai)]:
    for model_name, meta in provider_config.models.items():
        tools_str = "yes" if meta.supports_tools else "no"
        print(
            f"{model_name:<35} "
            f"{meta.context_window:>10,} "
            f"{meta.max_output_tokens:>10,} "
            f"${meta.cost_input_per_1m:>8.2f} "
            f"${meta.cost_output_per_1m:>8.2f} "
            f"{tools_str:>6}"
        )

---
## 3. Using Model Metadata

Model metadata isn't just informational — agents use it to make decisions:
- Can this model use tools? Check `supports_tools`
- Can I send images? Check `supports_vision` and `input_modalities`
- How much will this cost? Check `cost_*_per_1m`
- Will my prompt fit? Check `context_window`

In [ ]:
# Simulate: pick the cheapest model that supports tools
all_models = []
for provider_name, config in [("anthropic", anthropic), ("openai", openai)]:
    for model_name, meta in config.models.items():
        all_models.append((provider_name, model_name, meta))

tool_capable = [(p, m, meta) for p, m, meta in all_models if meta.supports_tools]
cheapest = min(tool_capable, key=lambda x: x[2].cost_input_per_1m)

print(f"Cheapest tool-capable model:")
print(f"  Provider: {cheapest[0]}")
print(f"  Model:    {cheapest[1]}")
print(f"  Cost:     ${cheapest[2].cost_input_per_1m}/1M input tokens")

In [ ]:
# Simulate: check if a prompt fits
sonnet = anthropic.models["claude-sonnet-4-20250514"]

estimated_tokens = 150_000  # big prompt
if estimated_tokens < sonnet.context_window:
    remaining = sonnet.context_window - estimated_tokens
    print(f"Prompt fits! {estimated_tokens:,} / {sonnet.context_window:,} tokens used")
    print(f"Room for {remaining:,} more tokens")
else:
    print(f"Prompt too large! {estimated_tokens:,} > {sonnet.context_window:,}")

In [ ]:
# Simulate: estimate cost for a batch of calls
n_calls = 1000
avg_input = 2000   # tokens per call
avg_output = 500   # tokens per call

model_meta = anthropic.models["claude-sonnet-4-20250514"]
total_input = n_calls * avg_input
total_output = n_calls * avg_output

input_cost = (total_input / 1_000_000) * model_meta.cost_input_per_1m
output_cost = (total_output / 1_000_000) * model_meta.cost_output_per_1m
total_cost = input_cost + output_cost

print(f"Cost estimate for {n_calls:,} calls:")
print(f"  Input:  {total_input:>10,} tokens x ${model_meta.cost_input_per_1m}/1M = ${input_cost:.2f}")
print(f"  Output: {total_output:>10,} tokens x ${model_meta.cost_output_per_1m}/1M = ${output_cost:.2f}")
print(f"  Total:  ${total_cost:.2f}")

---
## 4. Security: Path Traversal Prevention

Because `load_provider_config()` takes a string and turns it into a file path, we MUST validate it. Otherwise an attacker could pass `../../etc/passwd` as a provider name.

**Decision D-035**: Strict regex validation — `^[a-z][a-z0-9\-]*$`, max 64 characters.

This satisfies NIST 800-53 AC-3 (access enforcement) and OWASP A01 (broken access control).

In [ ]:
# Path traversal attack — BLOCKED
try:
    load_provider_config("../../etc/passwd")
except ArcLLMConfigError as e:
    print(f"BLOCKED: {e}")

In [ ]:
# Various attack vectors — all blocked
attacks = [
    ("../evil", "parent directory traversal"),
    ("pro/vider", "slash in name"),
    (".hidden", "dot-prefixed"),
    ("a@b", "special characters"),
    ("a b", "spaces"),
    ("", "empty string"),
    ("Anthropic", "uppercase (case-sensitive match)"),
    ("a" * 65, "name too long (>64 chars)"),
]

for attack, description in attacks:
    try:
        load_provider_config(attack)
        print(f"  FAIL - {description}: should have been blocked!")
    except ArcLLMConfigError:
        display_name = attack if len(attack) <= 30 else attack[:27] + "..."
        print(f"  BLOCKED - {description}: {display_name!r}")

In [ ]:
# Valid provider names that DO work
from arcllm.config import _validate_provider_name

valid_names = ["anthropic", "openai", "ollama", "my-custom-provider", "o1", "gpt4"]
for name in valid_names:
    try:
        _validate_provider_name(name)
        print(f"  VALID: {name!r}")
    except ArcLLMConfigError as e:
        print(f"  REJECTED: {name!r} - {e}")

---
## 5. Error Handling: Fail-Fast on Bad Config

Config errors are caught at load time, not during an LLM call. This is critical for unattended agents — you want to crash at startup, not 3 hours into a batch job.

In [ ]:
# Missing provider file
try:
    load_provider_config("nonexistent")
except ArcLLMConfigError as e:
    print(f"Missing file caught: {e}")

In [ ]:
# Malformed TOML (simulated with unittest.mock)
from unittest.mock import patch
from pathlib import Path
import tempfile

with tempfile.TemporaryDirectory() as tmp:
    tmp_path = Path(tmp)

    # Write a broken TOML file
    bad_toml = tmp_path / "config.toml"
    bad_toml.write_text("[unclosed section")

    with patch("arcllm.config._get_config_dir", return_value=tmp_path):
        try:
            load_global_config()
        except ArcLLMConfigError as e:
            print(f"Malformed TOML caught: {e}")

In [ ]:
# Wrong types in TOML (temperature should be float, not string)
with tempfile.TemporaryDirectory() as tmp:
    tmp_path = Path(tmp)

    bad_types = tmp_path / "config.toml"
    bad_types.write_text('[defaults]\nprovider = "ok"\ntemperature = "not-a-float"\n')

    with patch("arcllm.config._get_config_dir", return_value=tmp_path):
        try:
            load_global_config()
        except ArcLLMConfigError as e:
            print(f"Type validation caught: {e}")

In [ ]:
# All config errors are ArcLLMConfigError -> ArcLLMError
# So you can catch them at whatever granularity you need
try:
    load_provider_config("nonexistent")
except ArcLLMConfigError:
    print("Caught with ArcLLMConfigError (specific)")

try:
    load_provider_config("nonexistent")
except Exception as e:
    from arcllm import ArcLLMError
    print(f"Is ArcLLMError? {isinstance(e, ArcLLMError)}")

---
## 6. Config Models in Detail

Let's look at each Pydantic config model and how they compose.

In [ ]:
# DefaultsConfig — has sensible defaults for every field
# You can create one with no arguments and get reasonable values
defaults_empty = DefaultsConfig()
print(f"DefaultsConfig (no args):")
print(f"  provider: {defaults_empty.provider}")
print(f"  temperature: {defaults_empty.temperature}")
print(f"  max_tokens: {defaults_empty.max_tokens}")

In [ ]:
# ModuleConfig — extra='allow' is the magic
# It accepts any extra fields without complaining
basic = ModuleConfig(enabled=True)
print(f"Basic: enabled={basic.enabled}")

# With extra fields (module-specific settings)
budget_mod = ModuleConfig(enabled=True, monthly_limit_usd=1000.0, alert_threshold=0.8)
print(f"Budget: enabled={budget_mod.enabled}, limit=${budget_mod.monthly_limit_usd}, alert={budget_mod.alert_threshold}")

# The extra fields show up in model_dump too
print(f"Dump: {budget_mod.model_dump()}")

In [ ]:
# ModelMetadata — everything you need to know about a model
# All fields are required (no guessing in production)
from pydantic import ValidationError

try:
    ModelMetadata(context_window=100000)  # missing required fields
except ValidationError as e:
    print(f"Missing fields caught ({e.error_count()} errors):")
    for err in e.errors():
        print(f"  Missing: {err['loc'][0]}")

In [ ]:
# ProviderSettings — connection info for making API calls
# Note: api_key_env is the ENV VAR NAME, never the actual key
ps = anthropic.provider
print(f"To connect to Anthropic:")
print(f"  POST to: {ps.base_url}")
print(f"  Format:  {ps.api_format}")
print(f"  API key from: os.environ['{ps.api_key_env}']")
print(f"  Default model: {ps.default_model}")
print(f"\nThe key itself is NEVER in config files — security first.")

---
## 7. Config Discovery

Config files are discovered **package-relative** via `Path(__file__).parent`.

This means configs work in both:
- Development mode (`pip install -e .`)
- Installed mode (`pip install arcllm`)

In [ ]:
from arcllm.config import _get_config_dir

config_dir = _get_config_dir()
print(f"Config directory: {config_dir}")
print(f"\nFiles found:")
for f in sorted(config_dir.rglob("*.toml")):
    print(f"  {f.relative_to(config_dir)}")

In [ ]:
# Let's read the raw TOML to see what the loader parses
import tomllib

config_path = config_dir / "config.toml"
with open(config_path, "rb") as f:
    raw_data = tomllib.load(f)

print("Raw config.toml data:")
print(json.dumps(raw_data, indent=2, default=str))

In [ ]:
# And a provider TOML
provider_path = config_dir / "providers" / "anthropic.toml"
with open(provider_path, "rb") as f:
    raw_provider = tomllib.load(f)

print("Raw anthropic.toml data:")
print(json.dumps(raw_provider, indent=2, default=str))

---
## 8. Switching Providers

One of the key benefits of config-driven design: switching providers is just loading a different TOML.

In [ ]:
# Load both and compare
providers = {}
for name in ["anthropic", "openai"]:
    providers[name] = load_provider_config(name)

print(f"{'Setting':<20} {'Anthropic':<30} {'OpenAI':<30}")
print("-" * 80)
print(f"{'API format':<20} {providers['anthropic'].provider.api_format:<30} {providers['openai'].provider.api_format:<30}")
print(f"{'Base URL':<20} {providers['anthropic'].provider.base_url:<30} {providers['openai'].provider.base_url:<30}")
print(f"{'API key env':<20} {providers['anthropic'].provider.api_key_env:<30} {providers['openai'].provider.api_key_env:<30}")
print(f"{'Default model':<20} {providers['anthropic'].provider.default_model:<30} {providers['openai'].provider.default_model:<30}")
print(f"{'# of models':<20} {len(providers['anthropic'].models):<30} {len(providers['openai'].models):<30}")

In [ ]:
# Pattern: agent selects provider based on global defaults
global_cfg = load_global_config()
default_provider = global_cfg.defaults.provider

provider_cfg = load_provider_config(default_provider)
default_model = provider_cfg.provider.default_model
model_meta = provider_cfg.models[default_model]

print(f"Default provider: {default_provider}")
print(f"Default model:    {default_model}")
print(f"Context window:   {model_meta.context_window:,} tokens")
print(f"Supports tools:   {model_meta.supports_tools}")
print(f"\nThis is exactly what load_model() will do in Step 6.")

---
## 9. Override Chain Preview

The merge strategy (Decision D-033) is: **args > provider TOML > global TOML**

This isn't implemented yet (it will be in `load_model()`), but here's how it will work conceptually.

In [ ]:
# Preview: how overrides will chain
global_temp = global_cfg.defaults.temperature        # 0.7 (from config.toml)
provider_temp = provider_cfg.provider.default_temperature  # 0.7 (from anthropic.toml)
user_temp = 0.0  # agent passes temperature=0 for deterministic output

print(f"Override chain for 'temperature':")
print(f"  Global default:   {global_temp}")
print(f"  Provider default: {provider_temp}  (overrides global)")
print(f"  User override:    {user_temp}  (overrides everything)")
print(f"  Final value:      {user_temp}  (last writer wins)")
print(f"\nImplemented in load_model() (Step 6).")

---
## 10. Putting It Together: What an Agent Startup Looks Like

Here's the pattern agents will use once everything is wired up.

In [ ]:
def agent_startup_preview():
    """What agent initialization will look like."""
    # 1. Load global config
    global_cfg = load_global_config()
    print(f"1. Global config loaded")
    print(f"   Default provider: {global_cfg.defaults.provider}")
    print(f"   Modules: {', '.join(m for m, c in global_cfg.modules.items() if c.enabled) or 'none enabled'}")

    # 2. Load provider config
    provider_name = global_cfg.defaults.provider
    provider_cfg = load_provider_config(provider_name)
    print(f"\n2. Provider config loaded: {provider_name}")
    print(f"   Base URL: {provider_cfg.provider.base_url}")
    print(f"   API key from: ${provider_cfg.provider.api_key_env}")

    # 3. Select model
    model_name = provider_cfg.provider.default_model
    model_meta = provider_cfg.models[model_name]
    print(f"\n3. Model selected: {model_name}")
    print(f"   Context: {model_meta.context_window:,} tokens")
    print(f"   Tools:   {model_meta.supports_tools}")
    print(f"   Vision:  {model_meta.supports_vision}")

    # 4. Check API key exists (security: env vars only)
    import os
    key_name = provider_cfg.provider.api_key_env
    has_key = key_name in os.environ
    print(f"\n4. API key check: {'FOUND' if has_key else 'MISSING'} (${key_name})")

    # 5. Ready (Step 6 will return an LLMProvider here)
    print(f"\n5. Agent ready! (load_model() will return provider in Step 6)")

agent_startup_preview()

---
## Summary

Step 2 built the **config layer**:

```
config.toml          ->  GlobalConfig (defaults + module toggles)
providers/*.toml      ->  ProviderConfig (connection settings + model metadata)
config.py            ->  Pydantic models + loaders + path traversal prevention
```

**Key design decisions:**
- TOML format (stdlib `tomllib`, zero dependencies)
- Pydantic validation on load (fail-fast for agents)
- Package-relative file discovery (`Path(__file__).parent`)
- Simple override chain: args > provider > global
- Strict regex on provider names (path traversal prevention)
- `extra='allow'` on ModuleConfig (extensible without schema changes)
- API keys referenced by env var name, never stored in config